In [1]:
import soundfile as sf
import os
import numpy as np
import librosa
import torch
from config import fft_num, win_shift
base_path = r"D:\\work\\speechEnhancement\\datasets\\voicebank_demand"
audio_name = "p232_001.wav"
noisy_audio_path = os.path.join(base_path, "noisy_testset_wav", audio_name)

feat_wav, _ = sf.read(noisy_audio_path)  # (27861,)
c = np.sqrt(len(feat_wav) / np.sum((feat_wav ** 2.0))) 
feat_wav = feat_wav * c         # (27861,)
feat_x = librosa.stft(feat_wav, n_fft=fft_num, hop_length=win_shift, window='hann').T  # (175, 161)  (seq_len, freq)

#compress
feat_x, phase_x = np.abs(feat_x)**0.5, np.angle(feat_x)

feat_x = torch.FloatTensor(feat_x)
phase_x = torch.FloatTensor(phase_x)
feat_x = torch.stack((feat_x * torch.cos(phase_x), feat_x * torch.sin(phase_x)), dim=0) # (2, 175, 161) 

In [2]:
import torchaudio
from config import win_size
feat_wav, _ = torchaudio.load(noisy_audio_path)
feat_wav = feat_wav.squeeze(0)
c = torch.sqrt(feat_wav.shape[0] / torch.sum((feat_wav ** 2.0)))
feat_wav = feat_wav * c

feat_x = torch.stft(feat_wav, n_fft=fft_num, hop_length=win_shift, win_length=win_size, window=torch.hann_window(win_size), return_complex=True).T
feat_x, phase_x = torch.sqrt(torch.abs(feat_x)), torch.angle(feat_x)
feat_x = torch.stack((feat_x * torch.cos(phase_x), feat_x * torch.sin(phase_x)), dim=0)
print(feat_x.shape)

torch.Size([2, 175, 161])


In [3]:
from einops import rearrange
a = torch.rand([2, 4, 3])
print(a)
a.tolist()

tensor([[[0.0778, 0.6491, 0.9342],
         [0.6439, 0.5502, 0.3870],
         [0.7725, 0.9569, 0.0739],
         [0.8421, 0.7535, 0.8938]],

        [[0.1509, 0.5552, 0.8699],
         [0.0474, 0.9863, 0.3719],
         [0.5595, 0.6657, 0.0784],
         [0.3618, 0.5430, 0.6348]]])


[[[0.07783550024032593, 0.6490828394889832, 0.9342090487480164],
  [0.6438584923744202, 0.5501828789710999, 0.3869602680206299],
  [0.7724840044975281, 0.956874668598175, 0.0738721489906311],
  [0.8420925736427307, 0.7535222172737122, 0.8937874436378479]],
 [[0.15085262060165405, 0.555207371711731, 0.869949460029602],
  [0.04738122224807739, 0.9863272905349731, 0.37191063165664673],
  [0.5594611167907715, 0.6657001972198486, 0.07840949296951294],
  [0.3618142604827881, 0.5430347323417664, 0.6347635984420776]]]

In [4]:
seq_len = [2,3,4]
a = [torch.rand([2, q, 3]) for q in seq_len]
print(a)

[tensor([[[0.8250, 0.6453, 0.2622],
         [0.6777, 0.9875, 0.7710]],

        [[0.9462, 0.9748, 0.6558],
         [0.6576, 0.6944, 0.7657]]]), tensor([[[0.8245, 0.3624, 0.3872],
         [0.0555, 0.7249, 0.3516],
         [0.8215, 0.1983, 0.1448]],

        [[0.2362, 0.0973, 0.5680],
         [0.6008, 0.7743, 0.7779],
         [0.8992, 0.9939, 0.8197]]]), tensor([[[0.5122, 0.5664, 0.1788],
         [0.0824, 0.9181, 0.1125],
         [0.5145, 0.6901, 0.9491],
         [0.8213, 0.9827, 0.2220]],

        [[0.7499, 0.0512, 0.9023],
         [0.0473, 0.0482, 0.1076],
         [0.6532, 0.0838, 0.4200],
         [0.3294, 0.3424, 0.6944]]])]


In [10]:
b = [rearrange(a_, "N L C -> L (N C)") for a_ in a]
from torch.nn.utils.rnn import pad_sequence
c = pad_sequence(b, batch_first=True)
print(c.shape)
d = rearrange(c, "B L (N C) -> B N L C", N=2)
print(d.shape)
print(d)

torch.Size([3, 4, 6])
torch.Size([3, 2, 4, 3])
tensor([[[[0.8250, 0.6453, 0.2622],
          [0.6777, 0.9875, 0.7710],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[0.9462, 0.9748, 0.6558],
          [0.6576, 0.6944, 0.7657],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]]],


        [[[0.8245, 0.3624, 0.3872],
          [0.0555, 0.7249, 0.3516],
          [0.8215, 0.1983, 0.1448],
          [0.0000, 0.0000, 0.0000]],

         [[0.2362, 0.0973, 0.5680],
          [0.6008, 0.7743, 0.7779],
          [0.8992, 0.9939, 0.8197],
          [0.0000, 0.0000, 0.0000]]],


        [[[0.5122, 0.5664, 0.1788],
          [0.0824, 0.9181, 0.1125],
          [0.5145, 0.6901, 0.9491],
          [0.8213, 0.9827, 0.2220]],

         [[0.7499, 0.0512, 0.9023],
          [0.0473, 0.0482, 0.1076],
          [0.6532, 0.0838, 0.4200],
          [0.3294, 0.3424, 0.6944]]]])


In [8]:
b1 = []
for a_ in a:
    b1.append(torch.cat([a_[0], a_[1]], dim=1))

[b1[i] == b[i] for i in range(3)]

[tensor([[True, True, True, True, True, True],
         [True, True, True, True, True, True]]),
 tensor([[True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True]]),
 tensor([[True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True]])]